In [3]:
#Imports
import numpy as np
import pandas as pd
from os.path import join

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import loggers as pl_loggers

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import BertModel
from utils.data import RelevantDataset

#Notebook Parameters
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
bert_id = "google/bert_uncased_L-2_H-128_A-2"

seed_everything(1337)

Global seed set to 1337


1337

# Training

## Set Architecture

In [4]:
from utils.architectures import RelevantModuleV00 as RelevantModule


## Load Datasets

In [6]:
train_ds = RelevantDataset()
validation_ds = IsRelevantDataset(validation_join, device = device, dimensions = train_ds.dimensions)

## Hyperparameters

In [7]:
start_lr = 1e-4
batch_size = 16

In [9]:
# model
model = RelevantModule(
    bert = BertModel.from_pretrained(bert_id).to(device),
    input_size = sum(train_ds.dimensions[0][1]), 
    output_size = train_ds.dimensions[1],
    start_lr=start_lr
)

architecture_name = str(type(model))

#monitoring
tb_logger = pl_loggers.TensorBoardLogger('logs', name=architecture_name)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'logs/{architecture_name}/{tb_logger.log_dir}',
    verbose=True,
    save_last=True,
    save_top_k=-1, #save all
    mode="min",
    filename='-{epoch:02d}-{val_loss:.2f}'
)

#Dataloaders
train_dl = DataLoader(train_ds,batch_size  = batch_size, shuffle=True)
validation_dl = DataLoader(train_ds, batch_size  = 64, shuffle=False)

# training
trainer = Trainer(
    gpus=1, 
    precision=16, 
    logger=tb_logger, 
    callbacks=[checkpoint_callback]
)

Missing logger folder: logs\<class 'utils.architectures.RelevantModuleV00'>
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


## Executing run

In [ ]:
trainer.fit(model, train_dl, validation_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params
-------------------------------------------------
0 | bert              | BertModel  | 4.4 M 
1 | linear_after_bert | Linear     | 33.0 K
2 | feed_forward      | Sequential | 713 K 
-------------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.528    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Notes

Visit: https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html#restoring-training-state